<a href="https://colab.research.google.com/github/csy99/dna-nn-theory/blob/master/kmer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!curl https://cog-uk.s3.climb.ac.uk/2020-05-08/cog_2020-05-08_alignment.fasta -o cog_2020-05-08_alignment.fasta
!curl https://cog-uk.s3.climb.ac.uk/2020-05-08/cog_2020-05-08_metadata.csv -o metadata.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  407M  100  407M    0     0  19.7M      0  0:00:20  0:00:20 --:--:-- 22.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  915k  100  915k    0     0   464k      0  0:00:01  0:00:01 --:--:--  464k


In [1]:
!pip install biopython

     |████████████████████████████████| 2.3MB 5.1MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from Bio import SeqIO

In [3]:
from itertools import product
import re

In [4]:
word_size = 3
words = [''.join(c) for c in product('ATCGN', repeat=word_size)]
word_to_idx = {word: i for i, word in enumerate(words)}
word_to_idx

{'AAA': 0,
 'AAC': 2,
 'AAG': 3,
 'AAN': 4,
 'AAT': 1,
 'ACA': 10,
 'ACC': 12,
 'ACG': 13,
 'ACN': 14,
 'ACT': 11,
 'AGA': 15,
 'AGC': 17,
 'AGG': 18,
 'AGN': 19,
 'AGT': 16,
 'ANA': 20,
 'ANC': 22,
 'ANG': 23,
 'ANN': 24,
 'ANT': 21,
 'ATA': 5,
 'ATC': 7,
 'ATG': 8,
 'ATN': 9,
 'ATT': 6,
 'CAA': 50,
 'CAC': 52,
 'CAG': 53,
 'CAN': 54,
 'CAT': 51,
 'CCA': 60,
 'CCC': 62,
 'CCG': 63,
 'CCN': 64,
 'CCT': 61,
 'CGA': 65,
 'CGC': 67,
 'CGG': 68,
 'CGN': 69,
 'CGT': 66,
 'CNA': 70,
 'CNC': 72,
 'CNG': 73,
 'CNN': 74,
 'CNT': 71,
 'CTA': 55,
 'CTC': 57,
 'CTG': 58,
 'CTN': 59,
 'CTT': 56,
 'GAA': 75,
 'GAC': 77,
 'GAG': 78,
 'GAN': 79,
 'GAT': 76,
 'GCA': 85,
 'GCC': 87,
 'GCG': 88,
 'GCN': 89,
 'GCT': 86,
 'GGA': 90,
 'GGC': 92,
 'GGG': 93,
 'GGN': 94,
 'GGT': 91,
 'GNA': 95,
 'GNC': 97,
 'GNG': 98,
 'GNN': 99,
 'GNT': 96,
 'GTA': 80,
 'GTC': 82,
 'GTG': 83,
 'GTN': 84,
 'GTT': 81,
 'NAA': 100,
 'NAC': 102,
 'NAG': 103,
 'NAN': 104,
 'NAT': 101,
 'NCA': 110,
 'NCC': 112,
 'NCG': 113,
 'NCN'

In [5]:
vocab_size = 5 ** word_size
word_to_idx_func = np.vectorize(lambda word: word_to_idx[word], otypes=[np.int8])

In [8]:
with open('cog_2020-05-08_alignment.fasta', 'r') as f:
  # records = []
  # for seq in SeqIO.parse(f, 'fasta'):
    # records.append(" ".join([c for c in seq]))
  # records = np.array(records)
  records = np.array([record.seq._data.upper() for record in SeqIO.parse(f, 'fasta')])
seq_len = len(records[0])
print('Number of sequences :', len(records))
print('Length of sequences :', seq_len)

Number of sequences : 14277
Length of sequences : 29903


This process took 2 mins. 

In [9]:
records[0]

'------------------------------------------------------...-------------------------------------------------------------------'

In [10]:
covid_df = pd.read_csv("metadata.csv")
label = covid_df["adm1"]
label_names = covid_df["adm1"].unique()
label_names

array(['UK-NIR', 'UK-ENG', 'UK-SCT', 'UK-WLS'], dtype=object)

In [11]:
print(covid_df["adm1"].value_counts())
print(covid_df[:2000]["adm1"].value_counts())
print(covid_df[-1000:]["adm1"].value_counts())

UK-ENG    12281
UK-WLS     2332
UK-SCT     1640
UK-NIR      127
Name: adm1, dtype: int64
UK-ENG    916
UK-WLS    709
UK-SCT    313
UK-NIR     62
Name: adm1, dtype: int64
UK-ENG    561
UK-WLS    225
UK-SCT    214
Name: adm1, dtype: int64


In [ ]:
# def getWeight(label_names):
#   return 1000.0/covid_df["adm1"].value_counts()[label_names]

# weights = []
# for name in label_names:
#   weight = getWeight(name)
#   weights.append(weight)

# weights

In [12]:
for i in range(len(label_names)):
  label = label.replace(label_names[i], i)

In [14]:
# should change the training/validation set size later
TRAIN_SIZE = 2000
Test_SIZE = 1000
xtrain = records[:TRAIN_SIZE]
ytrain = label[:TRAIN_SIZE]
xtest = records[-Test_SIZE:]
ytest = label[-Test_SIZE:]

In [15]:
xtrain = xtrain.reshape(xtrain.shape[0], 1)
xtrain.shape

(2000, 1)

In [58]:
xtest = xtest.reshape(xtest.shape[0], 1)

In [27]:
xtrain[:6]

array([['------------------------------------------------------...-------------------------------------------------------------------'],
       ['------------------------------------------------------...-------------------------------------------------------------------'],
       ['------------------------------------------------------...-------------------------------------------------------------------'],
       ['------------------------------------------------------...-------------------------------------------------------------------'],
       ['------------------------------------------------------...-------------------------------------------------------------------'],
       ['------------------------------------------------------...-------------------------------------------------------------------']],
      dtype='<U29903')

In [29]:
for x in xtrain[:10]:
  print(len(x[0]))

29903
29903
29903
29903
29903
29903
29903
29903
29903
29903


In [36]:
def x_to_encoding(x, word_size=3, region_size=2, expand=True, truncate=500):
  x = tf.strings.regex_replace(x, "-", "")
  x = tf.strings.regex_replace(x, 'K', 'N')
  x = tf.strings.regex_replace(x, 'Y', 'N')
  x = tf.strings.regex_replace(x, 'R', 'N')
  x = tf.strings.regex_replace(x, 'W', 'N')
  x = tf.strings.regex_replace(x, 'M', 'N')
  x = tf.strings.regex_replace(x, 'S', 'N')
  x = x.numpy()[0].decode("utf-8") 
  # truncate for 2 reasons: data cleaning and reduce parameters
  x = x[:truncate]
  # print("len of x after preprocess", len(x))
  arr_x = [c for c in x]
  words = tf.strings.ngrams(arr_x, ngram_width=word_size, separator='').numpy()
  idx = word_to_idx_func(words.astype('U'))
  encoding = np.zeros((len(idx), vocab_size))
  encoding[range(len(idx)), idx] = 1
  # print(encoding.shape)
  res = np.hstack([np.roll(encoding, -shift, axis=0) for shift in range(region_size+1)])
  # print(res.shape)
  if expand:
    return np.expand_dims(res, axis=-1) 
  return res

In [50]:
region_size = 0
truncated_len = 100
xtrain2 = np.apply_along_axis(x_to_encoding, axis=1, \
                arr=xtrain, region_size=region_size, truncate=truncated_len)

In [51]:
xtrain2.shape

(2000, 98, 125, 1)

In [56]:
import gc
gc.collect()

9316

In [53]:
from tensorflow.keras import layers
shape = xtrain2.shape[1:]
model1 = keras.Sequential([
  layers.Conv2D(256, 3, activation="relu", input_shape=shape), 
  layers.MaxPooling2D(), 
  layers.Dropout(0.3), 
  layers.Flatten(), 
  layers.Dense(128, activation="relu"),
  layers.Dropout(0.5), 
  layers.Dense(4, activation="softmax"),
])

In [57]:
model1.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])
es_cb = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
hist1 = model1.fit(xtrain2, ytrain, epochs=15, callbacks=[es_cb])

Epoch 1/15
63/63 [==============================] - 9s 137ms/step - loss: 1.4603 - accuracy: 0.4150
Epoch 2/15
63/63 [==============================] - 8s 133ms/step - loss: 1.1540 - accuracy: 0.4450
Epoch 3/15
63/63 [==============================] - 8s 133ms/step - loss: 1.1493 - accuracy: 0.4565
Epoch 4/15
63/63 [==============================] - 8s 131ms/step - loss: 1.1495 - accuracy: 0.4525
Epoch 5/15
63/63 [==============================] - 8s 133ms/step - loss: 1.1417 - accuracy: 0.4540
Epoch 6/15
63/63 [==============================] - 8s 131ms/step - loss: 1.1505 - accuracy: 0.4495
Epoch 7/15
63/63 [==============================] - 8s 131ms/step - loss: 1.1475 - accuracy: 0.4575
Epoch 8/15
63/63 [==============================] - 8s 132ms/step - loss: 1.1575 - accuracy: 0.4590
Epoch 9/15
63/63 [==============================] - 8s 131ms/step - loss: 1.1418 - accuracy: 0.4555
Epoch 10/15
63/63 [==============================] - 8s 134ms/step - loss: 1.1364 - accuracy: 0.4535

In [60]:
xtest2 = np.apply_along_axis(x_to_encoding, axis=1, \
                arr=xtest, region_size=region_size, truncate=truncated_len)

In [62]:
model1.evaluate(xtest2, ytest)

32/32 [==============================] - 1s 38ms/step - loss: 1.0835 - accuracy: 0.5570


[1.0835148096084595, 0.5569999814033508]

In [ ]:
def evaluateModel(model, ds):
  ds_pred = np.argmax(model.predict(ds), axis=-1)
  print("prediction result shape", ds_pred.shape)
  ds_true = []
  label_ds = ds.map(lambda x, y: y)
  for item in label_ds:
    ds_true.append(item.numpy())
  ds_true = np.reshape(np.array(ds_true), (-1,)) 
  print("true label shape", ds_true.shape)
  return ds_pred, ds_true

In [ ]:
seq_ds_train_pred, seq_ds_train_true = evaluateModel(model1, seq_ds_train)

prediction result shape (2000,)
true label shape (2000,)


In [ ]:
import tensorflow_addons as tfa
confusion_mat = tfa.metrics.MultiLabelConfusionMatrix(num_classes=4)
confusion_mat.update_state(tf.one_hot(seq_ds_train_true, depth=4), tf.one_hot(seq_ds_train_pred, depth=4))

true negatives for class i in M(0,0)  
false positives for class i in M(0,1)  
false negatives for class i in M(1,0)  
true positives for class i in M(1,1)  

In [ ]:
# pd.set_option('display.float_format',lambda x : '%.2f' % x)
np.set_printoptions(suppress=True)
print('Confusion matrix:\n', confusion_mat.result().numpy())

Confusion matrix:
 [[[1922.   16.]
  [  52.   10.]]

 [[1084.    0.]
  [ 916.    0.]]

 [[1687.    0.]
  [ 313.    0.]]

 [[  26. 1265.]
  [   0.  709.]]]


In [ ]:
tmp = covid_df[:2000]["adm1"].value_counts()
# tmp["UK-NIR"]
for label in label_names:
  print(label, tmp[label])

UK-NIR 62
UK-ENG 916
UK-SCT 313
UK-WLS 709


In [ ]:
# seq_ds_test_pred, seq_ds_test_true = evaluateModel(model1, seq_ds_test)
# confusion_mat = tfa.metrics.MultiLabelConfusionMatrix(num_classes=4)
# confusion_mat.update_state(tf.one_hot(seq_ds_test_pred, depth=4), tf.one_hot(seq_ds_test_true, depth=4))
# print('Confusion matrix:\n', confusion_mat.result().numpy())

prediction result shape (1000,)
true label shape (1000,)
Confusion matrix:
 [[[999.   0.]
  [  1.   0.]]

 [[  0.   1.]
  [439. 560.]]

 [[786. 214.]
  [  0.   0.]]

 [[775. 225.]
  [  0.   0.]]]
